In [47]:
import string
import nltk
import re
import random
import pandas as pd
import numpy as np
from IPython.core.display import clear_output
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tag import pos_tag_sents
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist
from nltk import classify
from nltk import NaiveBayesClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfTransformer

In [33]:
financial_kaggle = pd.read_csv('financial_news.csv')
financial_kaggle

,Sentiment,Article
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...
...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...
4844,negative,Net sales of the Paper segment decreased to EU...


In [2]:
file = open("company_news.txt", "r")
lines = file.readlines()
print(len(lines))
lines[0]

309546


'MMM, Hot Stocks: XRX, MMM drop on results; ERIC, LOGI rally; BALY gets takeover bid, \xa0-\xa012 hours ago\n'

In [3]:
lines[11].split(",")

['MMM',
 ' Apple',
 ' Facebook',
 ' Fed Decision: 3 Things to Watch',
 ' \xa0-\xa0Jan 26',
 ' 2021\n']

In [4]:
new_lines = [x.split(",") for x in lines]
new_lines[0]

['MMM',
 ' Hot Stocks: XRX',
 ' MMM drop on results; ERIC',
 ' LOGI rally; BALY gets takeover bid',
 ' \xa0-\xa012 hours ago\n']

In [5]:
for i, x in enumerate(new_lines):
    if len(x) > 3:
        new_str = ""
        if 'hours ago' in x[len(x) - 1]:
            for j in range(1, len(x) - 1):
                new_str += x[j] + " "
            new_list = [x[0], new_str, x[len(x) - 1]]
        else:
            for j in range(1, len(x) - 2):
                new_str += x[j] + " "
            new_list =[x[0], new_str, x[len(x) - 2] + x[len(x) - 1]]
        new_lines[i] = new_list
    if i % 100000 == 0:
        print(i)

0
100000
200000
300000


In [6]:
new_lines[309545]

['ZTS',
 ' 4 Dependable Healthcare Stocks to Buy Hand Over Fist Right Now ',
 ' \xa0-\xa0Dec 01 2021\n']

In [7]:
articles = pd.DataFrame(new_lines, columns = ['Symbol', 'Title', 'Date'])
articles

,Symbol,Title,Date
0,MMM,Hot Stocks: XRX MMM drop on results; ERIC L...,- 12 hours ago\n
1,MMM,3M Gains as Price Hikes Cost Measures Offset...,- 17 hours ago\n
2,MMM,Asian Stocks Down as Investors Brace for Hawk...,- Jan 23 2022\n
3,MMM,Top 5 Things to Watch in Markets in The Week ...,- Jan 23 2022\n
4,MMM,U.S. stocks higher at close of trade; Dow Jon...,- Dec 22 2021\n
...,...,...,...
309541,ZTS,$0.97 Earnings Per Share Expected for Zoetis ...,- Dec 10 2021\n
309542,ZTS,Zoetis Raises FY22 Dividend By 30% Announces...,- Dec 08 2021\n
309543,ZTS,Zoetis announces $3.5 billion share buyback p...,- Dec 07 2021\n
309544,ZTS,Zoetis declares $0.325 dividend buyback plan,- Dec 07 2021\n


In [8]:
# fix dates in articles dataframe
articles['Date'] = articles['Date'].str.replace('-', '')
articles['Date'] = articles['Date'].str.strip()
for i in range(len(articles)):
    if 'ago' in articles.iloc[i]['Date']:
        x = i
        articles.iloc[i]['Date'] = 'Jan 25 2022'
    if i % 100000 == 0:
        print(i)
articles['Date'] = pd.to_datetime(articles['Date'], format = '%b %d %Y')
articles

0
100000
200000
300000


,Symbol,Title,Date
0,MMM,Hot Stocks: XRX MMM drop on results; ERIC L...,2022-01-25
1,MMM,3M Gains as Price Hikes Cost Measures Offset...,2022-01-25
2,MMM,Asian Stocks Down as Investors Brace for Hawk...,2022-01-23
3,MMM,Top 5 Things to Watch in Markets in The Week ...,2022-01-23
4,MMM,U.S. stocks higher at close of trade; Dow Jon...,2021-12-22
...,...,...,...
309541,ZTS,$0.97 Earnings Per Share Expected for Zoetis ...,2021-12-10
309542,ZTS,Zoetis Raises FY22 Dividend By 30% Announces...,2021-12-08
309543,ZTS,Zoetis announces $3.5 billion share buyback p...,2021-12-07
309544,ZTS,Zoetis declares $0.325 dividend buyback plan,2021-12-07


In [9]:
articles.iloc[0]

Symbol                                                  MMM
Title      Hot Stocks: XRX  MMM drop on results; ERIC  L...
Date                                    2022-01-25 00:00:00
Name: 0, dtype: object

In [35]:
df = articles.copy()
stop_words = list(stopwords.words('english'))
df['title/no_stopwords'] = df.apply(lambda row: pos_tag(word_tokenize(row['Title'])), axis = 1)
financial_kaggle['title/no_stopwords'] = financial_kaggle.apply(lambda row: pos_tag(word_tokenize(row['Article'])), axis = 1)
df

,Symbol,Title,Date,title/no_stopwords
0,MMM,Hot Stocks: XRX MMM drop on results; ERIC L...,2022-01-25,"[(Hot, JJ), (Stocks, NNS), (:, :), (XRX, NN), ..."
1,MMM,3M Gains as Price Hikes Cost Measures Offset...,2022-01-25,"[(3M, CD), (Gains, NNS), (as, IN), (Price, NNP..."
2,MMM,Asian Stocks Down as Investors Brace for Hawk...,2022-01-23,"[(Asian, JJ), (Stocks, NNP), (Down, NNP), (as,..."
3,MMM,Top 5 Things to Watch in Markets in The Week ...,2022-01-23,"[(Top, JJ), (5, CD), (Things, NNS), (to, TO), ..."
4,MMM,U.S. stocks higher at close of trade; Dow Jon...,2021-12-22,"[(U.S., NNP), (stocks, NNS), (higher, RBR), (a..."
...,...,...,...,...
309541,ZTS,$0.97 Earnings Per Share Expected for Zoetis ...,2021-12-10,"[($, $), (0.97, CD), (Earnings, NNP), (Per, NN..."
309542,ZTS,Zoetis Raises FY22 Dividend By 30% Announces...,2021-12-08,"[(Zoetis, NN), (Raises, VBZ), (FY22, NNP), (Di..."
309543,ZTS,Zoetis announces $3.5 billion share buyback p...,2021-12-07,"[(Zoetis, NNP), (announces, VBZ), ($, $), (3.5..."
309544,ZTS,Zoetis declares $0.325 dividend buyback plan,2021-12-07,"[(Zoetis, NN), (declares, VBZ), ($, $), (0.325..."


In [36]:
financial_kaggle

,Sentiment,Article,title/no_stopwords
0,neutral,"According to Gran , the company has no plans t...","[(According, VBG), (to, TO), (Gran, NNP), (,, ..."
1,neutral,Technopolis plans to develop in stages an area...,"[(Technopolis, NNP), (plans, VBZ), (to, TO), (..."
2,negative,The international electronic industry company ...,"[(The, DT), (international, JJ), (electronic, ..."
3,positive,With the new production plant the company woul...,"[(With, IN), (the, DT), (new, JJ), (production..."
4,positive,According to the company 's updated strategy f...,"[(According, VBG), (to, TO), (the, DT), (compa..."
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,"[(LONDON, NNP), (MarketWatch, NNP), (--, :), (..."
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,"[(Rinkuskiai, NNP), ('s, POS), (beer, NN), (sa..."
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"[(Operating, NN), (profit, NN), (fell, VBD), (..."
4844,negative,Net sales of the Paper segment decreased to EU...,"[(Net, JJ), (sales, NNS), (of, IN), (the, DT),..."


In [37]:
def lemmatize_sentence(tokens):
  lemmatizer = WordNetLemmatizer()
  sentence = []
  for word, tag in tokens:
    if tag.startswith('NN'):
      pos = 'n'
    elif tag.startswith('VB'):
      pos = 'v' 
    else:
      pos = 'a'
    sentence.append(lemmatizer.lemmatize(word, pos))
  return sentence

In [38]:
df['title/no_stopwords'] = df.apply(lambda row: lemmatize_sentence(row['title/no_stopwords']), axis = 1)
financial_kaggle['title/no_stopwords'] = financial_kaggle.apply(lambda row: lemmatize_sentence(row['title/no_stopwords']), axis = 1)
df

,Symbol,Title,Date,title/no_stopwords
0,MMM,Hot Stocks: XRX MMM drop on results; ERIC L...,2022-01-25,"[Hot, Stocks, :, XRX, MMM, drop, on, result, ;..."
1,MMM,3M Gains as Price Hikes Cost Measures Offset...,2022-01-25,"[3M, Gains, as, Price, Hikes, Cost, Measures, ..."
2,MMM,Asian Stocks Down as Investors Brace for Hawk...,2022-01-23,"[Asian, Stocks, Down, as, Investors, Brace, fo..."
3,MMM,Top 5 Things to Watch in Markets in The Week ...,2022-01-23,"[Top, 5, Things, to, Watch, in, Markets, in, T..."
4,MMM,U.S. stocks higher at close of trade; Dow Jon...,2021-12-22,"[U.S., stock, high, at, close, of, trade, ;, D..."
...,...,...,...,...
309541,ZTS,$0.97 Earnings Per Share Expected for Zoetis ...,2021-12-10,"[$, 0.97, Earnings, Per, Share, Expected, for,..."
309542,ZTS,Zoetis Raises FY22 Dividend By 30% Announces...,2021-12-08,"[Zoetis, Raises, FY22, Dividend, By, 30, %, An..."
309543,ZTS,Zoetis announces $3.5 billion share buyback p...,2021-12-07,"[Zoetis, announce, $, 3.5, billion, share, buy..."
309544,ZTS,Zoetis declares $0.325 dividend buyback plan,2021-12-07,"[Zoetis, declare, $, 0.325, dividend, buyback,..."


In [39]:
financial_kaggle

,Sentiment,Article,title/no_stopwords
0,neutral,"According to Gran , the company has no plans t...","[According, to, Gran, ,, the, company, have, n..."
1,neutral,Technopolis plans to develop in stages an area...,"[Technopolis, plan, to, develop, in, stage, an..."
2,negative,The international electronic industry company ...,"[The, international, electronic, industry, com..."
3,positive,With the new production plant the company woul...,"[With, the, new, production, plant, the, compa..."
4,positive,According to the company 's updated strategy f...,"[According, to, the, company, 's, updated, str..."
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,"[LONDON, MarketWatch, --, Share, price, end, l..."
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,"[Rinkuskiai, 's, beer, sale, fell, by, 6.5, pe..."
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"[Operating, profit, fell, to, EUR, 35.4, mn, f..."
4844,negative,Net sales of the Paper segment decreased to EU...,"[Net, sale, of, the, Paper, segment, decrease,..."


In [40]:
def remove_noise(reviews, stop_words = ()):

    cleaned_tokens = []

    for token in reviews:
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)


        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [41]:
df['title/no_stopwords'] = df.apply(lambda row: remove_noise(row['title/no_stopwords'], stop_words), axis = 1)
financial_kaggle['title/no_stopwords'] = financial_kaggle.apply(lambda row: remove_noise(row['title/no_stopwords'], stop_words), axis = 1)
df

,Symbol,Title,Date,title/no_stopwords
0,MMM,Hot Stocks: XRX MMM drop on results; ERIC L...,2022-01-25,"[hot, stocks, xrx, mmm, drop, result, eric, lo..."
1,MMM,3M Gains as Price Hikes Cost Measures Offset...,2022-01-25,"[3m, gains, price, hikes, cost, measures, offs..."
2,MMM,Asian Stocks Down as Investors Brace for Hawk...,2022-01-23,"[asian, stocks, investors, brace, hawkish, fed]"
3,MMM,Top 5 Things to Watch in Markets in The Week ...,2022-01-23,"[top, 5, things, watch, markets, week, ahead]"
4,MMM,U.S. stocks higher at close of trade; Dow Jon...,2021-12-22,"[u.s., stock, high, close, trade, dow, jones, ..."
...,...,...,...,...
309541,ZTS,$0.97 Earnings Per Share Expected for Zoetis ...,2021-12-10,"[0.97, earnings, per, share, expected, zoetis,..."
309542,ZTS,Zoetis Raises FY22 Dividend By 30% Announces...,2021-12-08,"[zoetis, raises, fy22, dividend, 30, announces..."
309543,ZTS,Zoetis announces $3.5 billion share buyback p...,2021-12-07,"[zoetis, announce, 3.5, billion, share, buybac..."
309544,ZTS,Zoetis declares $0.325 dividend buyback plan,2021-12-07,"[zoetis, declare, 0.325, dividend, buyback, plan]"


In [42]:
financial_kaggle

,Sentiment,Article,title/no_stopwords
0,neutral,"According to Gran , the company has no plans t...","[according, gran, company, plan, move, product..."
1,neutral,Technopolis plans to develop in stages an area...,"[technopolis, plan, develop, stage, area, less..."
2,negative,The international electronic industry company ...,"[international, electronic, industry, company,..."
3,positive,With the new production plant the company woul...,"[new, production, plant, company, would, incre..."
4,positive,According to the company 's updated strategy f...,"[according, company, 's, updated, strategy, ye..."
...,...,...,...
4841,negative,LONDON MarketWatch -- Share prices ended lower...,"[london, marketwatch, --, share, price, end, l..."
4842,neutral,Rinkuskiai 's beer sales fell by 6.5 per cent ...,"[rinkuskiai, 's, beer, sale, fell, 6.5, per, c..."
4843,negative,Operating profit fell to EUR 35.4 mn from EUR ...,"[operating, profit, fell, eur, 35.4, mn, eur, ..."
4844,negative,Net sales of the Paper segment decreased to EU...,"[net, sale, paper, segment, decrease, eur, 221..."


In [43]:
df['title/clean'] = df.apply(lambda row: ' '.join(row['title/no_stopwords']), axis = 1)
financial_kaggle['title/clean'] = financial_kaggle.apply(lambda row: ' '.join(row['title/no_stopwords']), axis = 1)
df

,Symbol,Title,Date,title/no_stopwords,title/clean
0,MMM,Hot Stocks: XRX MMM drop on results; ERIC L...,2022-01-25,"[hot, stocks, xrx, mmm, drop, result, eric, lo...",hot stocks xrx mmm drop result eric logi rally...
1,MMM,3M Gains as Price Hikes Cost Measures Offset...,2022-01-25,"[3m, gains, price, hikes, cost, measures, offs...",3m gains price hikes cost measures offset supp...
2,MMM,Asian Stocks Down as Investors Brace for Hawk...,2022-01-23,"[asian, stocks, investors, brace, hawkish, fed]",asian stocks investors brace hawkish fed
3,MMM,Top 5 Things to Watch in Markets in The Week ...,2022-01-23,"[top, 5, things, watch, markets, week, ahead]",top 5 things watch markets week ahead
4,MMM,U.S. stocks higher at close of trade; Dow Jon...,2021-12-22,"[u.s., stock, high, close, trade, dow, jones, ...",u.s. stock high close trade dow jones industri...
...,...,...,...,...,...
309541,ZTS,$0.97 Earnings Per Share Expected for Zoetis ...,2021-12-10,"[0.97, earnings, per, share, expected, zoetis,...",0.97 earnings per share expected zoetis inc. n...
309542,ZTS,Zoetis Raises FY22 Dividend By 30% Announces...,2021-12-08,"[zoetis, raises, fy22, dividend, 30, announces...",zoetis raises fy22 dividend 30 announces new b...
309543,ZTS,Zoetis announces $3.5 billion share buyback p...,2021-12-07,"[zoetis, announce, 3.5, billion, share, buybac...",zoetis announce 3.5 billion share buyback plan...
309544,ZTS,Zoetis declares $0.325 dividend buyback plan,2021-12-07,"[zoetis, declare, 0.325, dividend, buyback, plan]",zoetis declare 0.325 dividend buyback plan


In [44]:
def titles_for_model(cleaned_title):
  for title in cleaned_title:
    yield dict([token, True] for token in title)
    
positive = titles_for_model(financial_kaggle[financial_kaggle['Sentiment'] == 'positive']['title/no_stopwords'])
negative = titles_for_model(financial_kaggle[financial_kaggle['Sentiment'] == 'negative']['title/no_stopwords'])
neutral = titles_for_model(financial_kaggle[financial_kaggle['Sentiment'] == 'neutral']['title/no_stopwords'])

In [45]:
positive_data = [(title, 'Positive') for title in positive]
negative_data = [(title, 'Negative') for title in negative]
neutral_data = [(title, 'Neutral') for title in neutral]
complete_data = positive_data + negative_data + neutral_data

In [48]:
random.shuffle(complete_data)
training = complete_data[:round(len(complete_data) * .80)]
testing = complete_data[round(len(complete_data) * .80):]

In [49]:
nbclassifier = NaiveBayesClassifier.train(training)
print("Accuracy is: ", classify.accuracy(nbclassifier, testing))

Accuracy is:  0.6707946336429309


In [59]:
sample = "Asian Stocks Down as Investors Brace for Hawkish Fed"
custom_tokens = remove_noise(word_tokenize(sample))
print(custom_tokens)
print(nbclassifier.classify(dict([token, True] for token in custom_tokens)))

['asian', 'stocks', 'down', 'as', 'investors', 'brace', 'for', 'hawkish', 'fed']
Positive


In [64]:
titles = df['title/no_stopwords'].to_list()
df['Sentiment'] = ''
for i, t in enumerate(titles):
    df.at[i, 'Sentiment'] = nbclassifier.classify(dict([token, True] for token in t))
df    

,Symbol,Title,Date,title/no_stopwords,title/clean,Sentiment
0,MMM,Hot Stocks: XRX MMM drop on results; ERIC L...,2022-01-25,"[hot, stocks, xrx, mmm, drop, result, eric, lo...",hot stocks xrx mmm drop result eric logi rally...,Negative
1,MMM,3M Gains as Price Hikes Cost Measures Offset...,2022-01-25,"[3m, gains, price, hikes, cost, measures, offs...",3m gains price hikes cost measures offset supp...,Positive
2,MMM,Asian Stocks Down as Investors Brace for Hawk...,2022-01-23,"[asian, stocks, investors, brace, hawkish, fed]",asian stocks investors brace hawkish fed,Positive
3,MMM,Top 5 Things to Watch in Markets in The Week ...,2022-01-23,"[top, 5, things, watch, markets, week, ahead]",top 5 things watch markets week ahead,Neutral
4,MMM,U.S. stocks higher at close of trade; Dow Jon...,2021-12-22,"[u.s., stock, high, close, trade, dow, jones, ...",u.s. stock high close trade dow jones industri...,Negative
...,...,...,...,...,...,...
309541,ZTS,$0.97 Earnings Per Share Expected for Zoetis ...,2021-12-10,"[0.97, earnings, per, share, expected, zoetis,...",0.97 earnings per share expected zoetis inc. n...,Negative
309542,ZTS,Zoetis Raises FY22 Dividend By 30% Announces...,2021-12-08,"[zoetis, raises, fy22, dividend, 30, announces...",zoetis raises fy22 dividend 30 announces new b...,Neutral
309543,ZTS,Zoetis announces $3.5 billion share buyback p...,2021-12-07,"[zoetis, announce, 3.5, billion, share, buybac...",zoetis announce 3.5 billion share buyback plan...,Positive
309544,ZTS,Zoetis declares $0.325 dividend buyback plan,2021-12-07,"[zoetis, declare, 0.325, dividend, buyback, plan]",zoetis declare 0.325 dividend buyback plan,Neutral


In [17]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['title/clean'].tolist())
# tfidf = vectorizer.fit_transform(articles['Title'].tolist())
word_frequency = pd.DataFrame(tfidf[0].T.todense(), index = vectorizer.get_feature_names(), columns = ['TF-IDF'])
word_frequency = word_frequency.sort_values(by = 'TF-IDF', ascending = False)
word_frequency.head()
#word_frequency[word_frequency['TF-IDF'] != 0]
#word_frequency.loc['abbvie']

,TF-IDF
xrx,0.396300
baly,0.396300
logi,0.396300
eric,0.353875
mmm,0.341467


In [34]:
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(financial_kaggle.iloc[4843]['Article'])

{'neg': 0.0, 'neu': 0.751, 'pos': 0.249, 'compound': 0.743}

In [43]:
df.iloc[160011]['title/clean']

'u.s. stock close low one day reach record-highs'